In [4]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')


microsoft_data_df= pd.read_csv('Microsoft_Data.csv')



microsoft_data_df.shape

print(microsoft_data_df.isnull().sum()) 

microsoft_data_missingvalues_df= pd.DataFrame({'Column_names':microsoft_data_df.columns, 
                                              'Missing_values_count':microsoft_data_df.isnull().sum(),
                                              'Percentage_Missingvalues':microsoft_data_df.isnull().sum()/microsoft_data_df.shape[0]*100})

microsoft_data_missingvalues_df

microsoft_data_df['HasDetections'].value_counts()  ## categorical variable count

microsoft_data_df['OsBuild']=microsoft_data_df['OsBuild'].astype('object') ## convert the int column to an object type

microsoft_data_df['OsBuild'].dtype ## to find the type of the column

microsoft_data_df['OsBuild'].value_counts() ## we see int64 as type that is for the count

cols_to_drop=list(microsoft_data_missingvalues_df[microsoft_data_missingvalues_df['Percentage_Missingvalues']>50]['Column_names'])

microsoft_data_df.drop(columns=cols_to_drop,inplace=True)## drop columns

## Dropping the output variable for train & test split

y=microsoft_data_df['HasDetections']

microsoft_data_df.drop(columns='HasDetections',inplace=True)

/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
##Columns to drop whose missing values are unique and has one unique value for all records


cols_to_drop_1=[]

for col in microsoft_data_df.columns:
    if microsoft_data_df[col].nunique()==microsoft_data_df.shape[0] or microsoft_data_df[col].nunique()==1:
        cols_to_drop_1.append(col)
        
cols_to_drop_1

microsoft_data_df.drop(columns=cols_to_drop_1,inplace=True)

## To convert all the columns to object except 'Census_SystemVolumeTotalCapacity' as the nature is continous

for col in microsoft_data_df.columns:
    if col!='Census_SystemVolumeTotalCapacity':
        microsoft_data_df[col]=microsoft_data_df[col].astype(object)
        

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test= train_test_split(microsoft_data_df,y,test_size=0.2,random_state=43)

# Filling the missing values

for col in X_train.columns:
    if X_train[col].dtype=='object':
        X_train[col]=X_train[col].fillna(X_train[col].mode()[0]).astype(object)
        X_test[col]=X_test[col].fillna(X_train[col].mode()[0]).astype(object)
    else:
        X_train[col]=X_train[col].fillna(X_train[col].mean())
        X_test[col]=X_test[col].fillna(X_train[col].mean())


In [8]:
from sklearn.preprocessing import LabelEncoder
import numpy as np


class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)

In [ ]:
le=LabelEncoderExt()

train_city=['Hyderbad', 'Bangalore', 'Chennai','Jaipur','Delhi']
test_city=['Hyderbad', 'Bangalore', 'Chennai','Jaipur','Delhi','Mumbai']

le.fit(train_city)  ##fit is for train data
le.transform(test_city) ## transform is for test data


#using the data,the below code 'fit'is used to find the labelled values,whereas the 'transform' is used to transform
##to OG data set.

for col in X_train.columns:
    le.fit(X_train[col])
    X_train[col]=le.transform(X_train[col])
    X_test[col]=le.transform(X_test[col])

X_train

In [ ]:
## creating logisticregression

from sklearn.linear_model import LogisticRegression

logreg= LogisticRegression()
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)

In [ ]:
## Creating the confusion matrix

comparisons_df=pd.DataFrame({'Actuals':y_test,'Predictions':y_pred})

from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score

confusion_matrix(y_test,y_pred)
precision_score(y_test,y_pred)
recall_score(y_test,y_pred)
f1_score(y_test,y_pred)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

multinomial=MultinomialNB()
multinomial.fit(X_train,y_train)
y_pred=multinomial.predict(X_test)



In [70]:
## Chi-sqaure test -- used to test the correlation between two categorical variable

import scipy.stats     

crosstab= pd.crosstab(microsoft_data_df['OsBuild'],y)

crosstab

result= scipy.stats.chi2_contingency(crosstab)

result ## ' 1.434114178500418e-17' as this is less than the 0.05 we reject he null hypothesis

print('p value is:',result[1])


p value is: 1.434114178500418e-17


In [40]:
object_col= [col for col in microsoft_data_df.columns if microsoft_data_df[col].dtype=='object']



In [74]:
p_values=[]
for col in object_col:
    crosstab_1=pd.crosstab(microsoft_data_df[col],microsoft_data_df['HasDetections'])
    result=scipy.stats.chi2_contingency(crosstab_1)
    p_values.append(result[1])
    


KeyError: 'MachineIdentifier'

In [43]:
p_values_df=pd.DataFrame({'Column Name':object_col,'p_values':p_values})

p_values_df=p_values_df.sort_values(by='p_values',ascending=False)

p_values_df

,Column Name,p_values
1,ProductName,6.701547e-01
7,OsVer,5.191060e-01
0,MachineIdentifier,4.980265e-01
27,Census_FlightRing,2.174158e-01
5,Platform,1.531325e-02
14,Census_DeviceFamily,2.054026e-03
25,Census_GenuineStateName,1.883395e-05
11,SkuEdition,3.911500e-06
15,Census_PrimaryDiskTypeName,2.541452e-13
24,Census_OSWUAutoUpdateOptionsName,1.077721e-14
